In [42]:
# Uncomment to upgrade packages
# !pip3 install pandas --upgrade --quiet
# !pip3 install numpy  --upgrade --quiet
# !pip3 install scipy --upgrade --quiet
# !pip3 install statsmodels  --upgrade --quiet
# !pip3 install seaborn  --upgrade --quiet
# !pip3 install matplotlib  --upgrade --quiet
# !pip3 install scikit-learn  --upgrade  --quiet
# !pip install scikit-optimize  --quiet
# !pip install -U --quiet yellowbrick
# !pip install apafib --upgrade  --quiet
# !pip install --upgrade pip
# !pip install ucimlrepo

In [43]:
# Imports de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns      
import missingno as msno
import statsmodels.api as sm
import matplotlib.dates as mdates

from apafib import load_dormir
from scipy import stats
from time import time
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy


import warnings
warnings.filterwarnings('ignore')

# Definiciones
RND = 16
MAX_ITER = 5000

sns.set(style="whitegrid", font_scale=1.05)

# Funciones auxiliares
def format_pval(p):
    if p == 0:
        return r"$0$"
    exp = int(np.floor(np.log10(p)))
    if exp >= -3:
        return f"{p:.4f}"                
    else:
        mant = p / (10.0**exp)
        return rf"${mant:.2f}\times10^{{{exp}}}$"
    
# quitar warnings:
import warnings
warnings.filterwarnings("ignore", message="findfont:.*")

init_time = time()

# Práctica de APA. Preprocesado de las variables.

## Introducción

Autores: Oriol Farrés y Marc Gil

Al tratarse de un problema de series temporales y disponer de los datos íntegramente de los años 2011 a 2024, dividiremos de la siguiente manera:

* Train Set: Años 2011 a 2020 (10 años)
* Test Set: Años 2021 a 2024 (4 años)

Es decir un ratio de ~71.5% para el conjunto de entrenamiento y un ~28.5% para el conjunto de test.

---

Al tener tal cantidad de trabajo con el preprocesado de las variables para el problema, para asegurarnos de que hay consistencia y evitar cometer errores metodológicos, seguiremos la siguiente estructura (basada en la lista propuesta en el LAB y en el guión de la práctica): 




**Fase A: Obtención de los datos**

1. Preámbulo

**Fase B: Limpieza de los datos**

1. Preparar las variables (Sanity Check)
2. Tratamiento de valores incoherentes o incorrectos
3. Tratamiento de missing values
4. Tratamiento de outliers
5. Crear nuevas variables que pueden ser interesantes (Feature engineering)
6. Re-nombrado de las variables
7. Eliminar data leakage

**Fase C: Separación de los datos**

1. División del dataset en train/test 

**Fase D: Análisis**

1. Exploratorio Mínimo

**Fase E: Preprocesado de  las variables**

1. Dividir dataset en X e y
2. Transformar categóricas con OHE y escalar numéricas
3. Aplicar cambios 

**Fase F: Reducción de la dimensionalidad y visualización**

1. Aplicar PCA
2. Aplicar t-SNE
3. Comentarios sobre la reducción de dimensionalidad  

<hr style="height:2px;border:none;color:red;background-color:blue;" />
<hr style="height:2px;border:none;color:red;background-color:red;" />
<hr style="height:2px;border:none;color:red;background-color:blue;" />

# Fase A-Obtención de los datos

## A.1-Preámbulo

> Tenemos todo el dataset preparado en el directorio data/raw/raw_atp_matches.csv.

In [44]:
tennis = pd.read_csv('./data/raw/raw_atp_matches.csv')
tennis.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,month,day,month_name,tourney_points
0,2011-339,Brisbane,Hard,32,A,20110102,1,104417,1.0,NaN,...,4.0,5.0,5580.0,173.0,309.0,2011,1,2,January,250
1,2011-339,Brisbane,Hard,32,A,20110102,2,103582,NaN,NaN,...,5.0,58.0,835.0,75.0,643.0,2011,1,2,January,250
2,2011-339,Brisbane,Hard,32,A,20110102,3,105051,NaN,Q,...,8.0,196.0,263.0,204.0,243.0,2011,1,2,January,250
3,2011-339,Brisbane,Hard,32,A,20110102,4,104797,8.0,NaN,...,3.0,40.0,1031.0,43.0,975.0,2011,1,2,January,250
4,2011-339,Brisbane,Hard,32,A,20110102,5,103888,4.0,NaN,...,6.0,16.0,1991.0,83.0,600.0,2011,1,2,January,250


In [45]:
tennis.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tourney_id,39541,1894,2011-580,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tourney_name,39541,1078,Roland Garros,1778,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surface,39488,4,Hard,23060,NaN,NaN,NaN,NaN,NaN,NaN,NaN
draw_size,39541.0,NaN,NaN,NaN,57.467489,42.144138,2.0,32.0,32.0,96.0,128.0
tourney_level,39541,6,A,21148,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tourney_date,39541.0,NaN,NaN,NaN,20174404.064313,41143.884302,20110102.0,20140319.0,20170731.0,20210726.0,20241218.0
match_num,39541.0,NaN,NaN,NaN,167.239802,151.260611,1.0,21.0,190.0,283.0,1701.0
winner_id,39541.0,NaN,NaN,NaN,117737.747477,29694.476989,100644.0,104607.0,105373.0,109739.0,212721.0
winner_seed,16884.0,NaN,NaN,NaN,7.543592,6.988957,1.0,3.0,5.0,9.0,33.0
winner_entry,5347,10,Q,2968,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<hr style="height:2px;border:none;color:red;background-color:blue;" />
<hr style="height:2px;border:none;color:red;background-color:red;" />
<hr style="height:2px;border:none;color:red;background-color:blue;" />

# Fase B-Limpieza de los datos

Antes de empezar con la limpieza, vamos a ver cuantas filas tenemos en el dataset, para asegurarnos de que no cometemos ningún error.

In [46]:
print(f"Nuestro dataset tiene {tennis.shape[1]} variables y {tennis.shape[0]:,} columnas.")

Nuestro dataset tiene 54 variables y 39,541 columnas.


## B.1-Preparar las variables (Sanity Check)

En este apartado, vamos a preparnos las variables para poder tratarlas correctamente en los siguientes pasos. Un primer paso es eliminar aquellos partidos donde el rival se ha retirado/no se ha presentado, ya que esto es, obviamente impredecible (almenos con nuestros datos), y solo confundiría al modelo.

### B.1-Paso 1. Eliminar partidos inválidos

In [47]:
tennis['score'].head()

0           6-2 6-4
1    1-6 7-6(3) 6-2
2       4-6 6-2 6-4
3        7-6(5) 6-4
4           6-1 6-4
Name: score, dtype: object

In [48]:
# 1. Ver los 20 resultados más comunes (para ver lo "normal")
print("Lo normal:")
print(tennis['score'].value_counts().head(10))

print("\n----------------\n")

# 2. TRUCO PRO: Buscar solo los scores que contienen letras
# Esto te mostrará de golpe todos los RET, W/O, Def, etc.
scores_con_letras = tennis[tennis['score'].str.contains('[a-zA-Z]', na=False)]['score']

print("Lo sospechoso (contiene letras):")
print(scores_con_letras.unique())

Lo normal:
score
6-3 6-4    1355
6-4 6-4    1330
6-3 6-2     849
6-4 6-3     803
6-4 6-2     787
6-3 6-3     728
6-2 6-4     632
6-2 6-2     548
6-2 6-3     499
6-1 6-4     432
Name: count, dtype: int64

----------------

Lo sospechoso (contiene letras):
['6-2 RET' '6-0 5-0 RET' '6-3 6-1 4-2 RET' '6-1 6-0 2-0 RET'
 '6-7(5) 6-2 6-1 4-3 RET' 'W/O' '5-2 RET' '6-2 1-0 RET' '6-1 0-1 RET'
 '3-6 6-5 RET' '4-0 RET' '6-1 2-0 RET' '5-0 RET' '6-7(5) 6-2 4-2 RET'
 '4-5 RET' '6-2 4-1 RET' '6-4 RET' '7-5 2-0 RET' '5-4 RET' '6-1 5-3 RET'
 '6-4 3-1 RET' '3-0 RET' '3-4 RET' '3-6 2-1 RET' '4-6 6-1 2-0 RET'
 '2-6 4-2 RET' '5-1 RET' '6-3 4-0 RET' '4-6 6-1 4-4 RET'
 '7-6(6) 3-6 5-2 RET' '6-2 3-0 RET' '6-7(5) 6-3 3-0 RET' '6-4 1-0 RET'
 '7-6(1) 1-0 RET' '6-2 2-0 RET' '4-6 6-3 6-2 3-1 RET' '6-4 6-1 2-3 RET'
 '4-3 RET' '6-4 6-6 RET' '7-6(5) 2-0 RET' '6-7(4) 6-4 3-0 RET'
 '7-6(5) 3-6 5-3 RET' '7-5 3-1 RET' '6-3 6-3 RET' '6-4 4-2 RET' '2-3 RET'
 '6-3 6-7(5) 6-2 1-1 RET' '6-7 6-3 6-1 RET' '2-6 6-2 6-3 2-0 RET'
 

In [49]:
# 1. Copia de seguridad inicial (opcional, pero recomendada)
print(f"Total partidos antes de limpiar score: {len(tennis)}")

# 2. Asegurar que la columna sea tipo string (texto) para que no falle el filtro
tennis['score'] = tennis['score'].astype(str)

# 3. Crear el filtro de "Marcadores Sucios"
# Buscamos: 'W/O' (No presentado), 'RET' (Retirado), 'Def' (Descalificado)
# case=False detecta tanto 'Ret' como 'RET'
filtro_score_sucio = tennis['score'].str.contains('W/O|RET|Def|Walkover|Aban|ABD', case=False, na=False)

# 4. Ver qué vamos a eliminar (para que te quedes tranquilo)
print("\nEjemplos de scores que vamos a ELIMINAR:")
print(tennis[filtro_score_sucio]['score'].unique()[:10]) # Mostramos los 10 primeros tipos

# 5. Aplicar el borrado
# El símbolo '~' significa "Quédate con lo que NO es sucio"
tennis = tennis[~filtro_score_sucio].copy()

# 6. Resetear el índice
tennis.reset_index(drop=True, inplace=True)

print(f"\nTotal partidos después de limpiar score: {len(tennis)}")

Total partidos antes de limpiar score: 39541

Ejemplos de scores que vamos a ELIMINAR:
['6-2 RET' '6-0 5-0 RET' '6-3 6-1 4-2 RET' '6-1 6-0 2-0 RET'
 '6-7(5) 6-2 6-1 4-3 RET' 'W/O' '5-2 RET' '6-2 1-0 RET' '6-1 0-1 RET'
 '3-6 6-5 RET']

Total partidos después de limpiar score: 38143


In [50]:
scores_con_letras = tennis[tennis['score'].str.contains('[a-zA-Z]', na=False)]['score']

print("Lo sospechoso (contiene letras):")
print(scores_con_letras.unique())

Lo sospechoso (contiene letras):
[]


---

### B.1-Paso 2. Tratar variables temporales

<hr style="height:2px;border:none;color:red;background-color:blue;" />
<hr style="height:2px;border:none;color:red;background-color:red;" />
<hr style="height:2px;border:none;color:red;background-color:blue;" />

# Tiempo total de ejecución

In [51]:
print(f"Total Running time {timedelta(seconds=(time() - init_time))}")

Total Running time 0:00:00.635471
